In [1]:
import os, sys
from dotenv import find_dotenv, load_dotenv
from fredapi import Fred
import pandas as pd
import pandas_datareader as pdr
import sqlite3
import logging
import re

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)
from data.db_helper import DB_Helper

%matplotlib inline
import matplotlib.pyplot as plt

load_dotenv(find_dotenv())
fred_api_key = os.environ.get('FRED_API_KEY')
fred = Fred(api_key=fred_api_key)

db = DB_Helper(os.environ.get('DB_PATH'))

In [2]:
ticker = 'AAPL'
series_ids = ['SP500', 'DGS10', 'DGS5', 'USD3MTD156N', 'USD1WKD156N', 'FF']

In [3]:
# adding stock series info
if db.have_data_for_series(ticker):
    print('have data for {0} - not downloading'.format(ticker))
else:
    ds = pdr.data.DataReader(ticker, 'yahoo')
    db.add_data_series_and_data_yahoo(ds, ticker)

have data for AAPL - not downloading


In [4]:
# Adding series info
for id in series_ids:
    if db.have_data_for_series(id):
        print('have data for {0} - not downloading'.format(id))
    else:
        ds_meta = fred.get_series_info(id)
        db.add_data_series_fred(ds_meta)
        
        ds_data = fred.get_series(id)
        db.add_data_fred(ds_data, id)

have data for SP500 - not downloading
have data for DGS10 - not downloading
have data for DGS5 - not downloading
have data for USD3MTD156N - not downloading
have data for USD1WKD156N - not downloading
have data for FF - not downloading


In [5]:
nasdaq_path = os.environ.get('NASDAQ_PATH')
db.add_companies_from_csv(nasdaq_path, 'NASDAQ')

nyse_path = os.environ.get('NYSE_PATH')
db.add_companies_from_csv(nyse_path, 'NYSE')

IntegrityError: UNIQUE constraint failed: company.symbol